# Operaciones con Base de Datos SQLite usando SQLAlchemy

Este notebook demuestra cómo interactuar con una base de datos SQLite usando SQLAlchemy en Python. La base de datos `bd_negocio2.db` contiene dos tablas: `negocios_uno` y `negocios_dos`, cargadas desde archivos CSV.

Cubriremos los siguientes temas, reforzando aspectos teóricos:

- **Realización de operaciones CRUD (Create, Read, Update, Delete) desde Python**
- **Consultas avanzadas en SQLAlchemy (JOINs, filtrado, agrupación)**
- **Optimización de consultas en bases de datos relacionales**
- **Ejercicio práctico: Optimización y consulta de grandes volúmenes de datos**

**Nota teórica general:** SQLAlchemy es un ORM (Object-Relational Mapping) que permite mapear clases Python a tablas de bases de datos, facilitando operaciones de alto nivel mientras mantiene la flexibilidad de SQL puro. Esto promueve el principio de "Don't Repeat Yourself" (DRY) y mejora la mantenibilidad del código.

## Configuración Inicial

Importamos las librerías necesarias y configuramos la conexión a la base de datos.

In [9]:
import sqlalchemy as db
from sqlalchemy import create_engine, Column, Integer, String, Date, Numeric, ForeignKey, Text, Float, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, joinedload, relationship
from sqlalchemy import text
from sqlalchemy.exc import SQLAlchemyError

import pandas as pd
from datetime import datetime

In [10]:
# Crear el motor de conexión a la base de datos SQLite
engine = create_engine('sqlite:///bd_negocio2.db', echo=True)  # echo=True para ver logs SQL

In [11]:
# Crear sesión para interactuar con la base de datos
Session = sessionmaker(bind=engine)
session = Session()

In [12]:
# Definir la base para modelos declarativos
Base = declarative_base()

## Definición de Modelos

Definimos modelos para ambas tablas asumiendo estructuras similares basadas en el esquema proporcionado. Si las tablas difieren, ajusta las columnas accordingly.

In [16]:
class NegociosUno(Base):
    __tablename__ = 'negocios_uno'  # Nombre de la tabla desde el script anterior
    __table_args__ = {'extend_existing': True}

    tipdoc = Column(String)
    numdoc = Column(String, primary_key=True)
    fcre = Column(Date)
    monto_mes_ini = Column(Float)
    cuota_mes_ini = Column(Float)
    ncuota_mes_ini = Column(Integer)
    tasa_mes_ini = Column(Float)
    saldo_mes_20151231 = Column(Float)
    Flg_Refin_Jud_Cast_6m = Column(Integer)
    per_perdida_6m = Column(String)
    SALDO_ANTES_PERD_6m = Column(Float)
    recuperado_flg_6m = Column(Integer)
    recuperado_periodo_6m = Column(Integer)
    fuga_periodo_6m = Column(Integer)
    fuga_periodo_6m_pp = Column(Float)
    ent_fuga_mes = Column(String)
    clas_ent_fuga_mes = Column(String)
    ent_fuga_pp = Column(String)
    clas_ent_fuga_pp = Column(String)
    target_6m = Column(Integer)
    flg_usar = Column(Integer)
    otro_mes_sf_act = Column(Float)
    otro_nentmes_sf_act = Column(Integer)
    otro_lineames_sf_act = Column(Float)
    otro_pp_sf_act = Column(Float)
    otro_nentpp_sf_act = Column(Integer)
    otro_lineapp_sf_act = Column(Float)
    otro_tc_sf_act = Column(Float)
    otro_nenttc_sf_act = Column(Integer)
    otro_lineatc_sf_act = Column(Float)
    otro_mes_sf_3um = Column(Float)
    otro_nentmes_sf_3um = Column(Integer)
    otro_lineames_sf_3um = Column(Float)
    otro_pp_sf_3um = Column(Float)
    otro_nentpp_sf_3um = Column(Integer)
    otro_lineapp_sf_3um = Column(Float)
    otro_tc_sf_3um = Column(Float)
    otro_nenttc_sf_3um = Column(Integer)
    otro_lineatc_sf_3um = Column(Float)
    otro_mes_sf_6um = Column(Float)
    otro_nentmes_sf_6um = Column(Integer)
    otro_lineames_sf_6um = Column(Float)
    otro_pp_sf_6um = Column(Float)
    otro_nentpp_sf_6um = Column(Integer)
    otro_lineapp_sf_6um = Column(Float)
    otro_tc_sf_6um = Column(Float)
    otro_nenttc_sf_6um = Column(Integer)
    otro_lineatc_sf_6um = Column(Float)
    sexo = Column(String)
    edad = Column(Integer)
    departamento = Column(String)
    estcivil = Column(String)
    gradoinst = Column(String)
    monto_mes_ult = Column(Float)
    cuota_mes_ult = Column(Float)
    ncuotas_mes_ult = Column(Integer)
    CuotasPagadas_mes_ult = Column(Integer)
    Tasa_mes_ult = Column(Float)
    fechadesembolso_ult = Column(Date)
    diasmora_mes_ult = Column(Integer)
    meses_ult_desemb_mes = Column(Integer)
    nro_desemb_mes = Column(Integer)
    monto_tot_mes = Column(Float)
    monto_prom_mes = Column(Float)
    nro_desemb_pp = Column(Integer)
    monto_tot_pp = Column(Float)
    monto_prom_pp = Column(Float)
    saldomes_peer_act = Column(Float)
    nent_me_peer_act = Column(Integer)
    ratio_mes_peer_sf_act = Column(Float)
    saldomes_peer_3um = Column(Float)
    nent_me_peer_3um = Column(Integer)
    ratio_mes_peer_sf_3um = Column(Float)
    saldomes_peer_6um = Column(Float)
    nent_me_peer_6um = Column(Integer)
    ratio_mes_peer_sf_6um = Column(Float)
    nprod_sf_act = Column(Integer)
    nprod_sf_3u = Column(Integer)
    nprod_sf_6u = Column(Integer)
    mesescte_csf = Column(Integer)
    mesescte_mes = Column(Integer)
    fingreso_csf = Column(Date)
    fingreso_mes = Column(Date)
    otrosprod_csf_act = Column(Integer)
    lineaotroprod_csf_act = Column(Float)
    otrosprod_csf_3um = Column(Integer)
    lineaotroprod_csf_3um = Column(Float)
    otrosprod_csf_6um = Column(Integer)
    lineaotroprod_csf_6um = Column(Float)
    ventas_mensual = Column(Float)
    patrimonio = Column(Float)
    tipolocal = Column(String)
    tipoactividad = Column(String)
    actividadeconomica = Column(String)
    tipovivienda = Column(String)
    r_cuotapagada = Column(Float)
    NodeID = Column(Integer)
    PredictedValue = Column(Integer)
    PredictedProbability_1 = Column(Float)
    PredictedProbability_2 = Column(Float)
    SampleAssignment = Column(String)

    def __repr__(self):
        return f"<NegociosUno(numdoc='{self.numdoc}', departamento='{self.departamento}')>"

class NegociosDos(Base):
    __tablename__ = 'negocios_dos'  # Nombre de la tabla desde el script anterior
    __table_args__ = {'extend_existing': True}

    tipdoc = Column(String)
    numdoc = Column(String, primary_key=True)
    fcre = Column(Date)
    monto_mes_ini = Column(Float)
    cuota_mes_ini = Column(Float)
    ncuota_mes_ini = Column(Integer)
    tasa_mes_ini = Column(Float)
    saldo_mes_20151231 = Column(Float)
    Flg_Refin_Jud_Cast_6m = Column(Integer)
    per_perdida_6m = Column(String)
    SALDO_ANTES_PERD_6m = Column(Float)
    recuperado_flg_6m = Column(Integer)
    recuperado_periodo_6m = Column(Integer)
    fuga_periodo_6m = Column(Integer)
    fuga_periodo_6m_pp = Column(Float)
    ent_fuga_mes = Column(String)
    clas_ent_fuga_mes = Column(String)
    ent_fuga_pp = Column(String)
    clas_ent_fuga_pp = Column(String)
    target_6m = Column(Integer)
    flg_usar = Column(Integer)
    otro_mes_sf_act = Column(Float)
    otro_nentmes_sf_act = Column(Integer)
    otro_lineames_sf_act = Column(Float)
    otro_pp_sf_act = Column(Float)
    otro_nentpp_sf_act = Column(Integer)
    otro_lineapp_sf_act = Column(Float)
    otro_tc_sf_act = Column(Float)
    otro_nenttc_sf_act = Column(Integer)
    otro_lineatc_sf_act = Column(Float)
    otro_mes_sf_3um = Column(Float)
    otro_nentmes_sf_3um = Column(Integer)
    otro_lineames_sf_3um = Column(Float)
    otro_pp_sf_3um = Column(Float)
    otro_nentpp_sf_3um = Column(Integer)
    otro_lineapp_sf_3um = Column(Float)
    otro_tc_sf_3um = Column(Float)
    otro_nenttc_sf_3um = Column(Integer)
    otro_lineatc_sf_3um = Column(Float)
    otro_mes_sf_6um = Column(Float)
    otro_nentmes_sf_6um = Column(Integer)
    otro_lineames_sf_6um = Column(Float)
    otro_pp_sf_6um = Column(Float)
    otro_nentpp_sf_6um = Column(Integer)
    otro_lineapp_sf_6um = Column(Float)
    otro_tc_sf_6um = Column(Float)
    otro_nenttc_sf_6um = Column(Integer)
    otro_lineatc_sf_6um = Column(Float)
    sexo = Column(String)
    edad = Column(Integer)
    departamento = Column(String)
    estcivil = Column(String)
    gradoinst = Column(String)
    monto_mes_ult = Column(Float)
    cuota_mes_ult = Column(Float)
    ncuotas_mes_ult = Column(Integer)
    CuotasPagadas_mes_ult = Column(Integer)
    Tasa_mes_ult = Column(Float)
    fechadesembolso_ult = Column(Date)
    diasmora_mes_ult = Column(Integer)
    meses_ult_desemb_mes = Column(Integer)
    nro_desemb_mes = Column(Integer)
    monto_tot_mes = Column(Float)
    monto_prom_mes = Column(Float)
    nro_desemb_pp = Column(Integer)
    monto_tot_pp = Column(Float)
    monto_prom_pp = Column(Float)
    saldomes_peer_act = Column(Float)
    nent_me_peer_act = Column(Integer)
    ratio_mes_peer_sf_act = Column(Float)
    saldomes_peer_3um = Column(Float)
    nent_me_peer_3um = Column(Integer)
    ratio_mes_peer_sf_3um = Column(Float)
    saldomes_peer_6um = Column(Float)
    nent_me_peer_6um = Column(Integer)
    ratio_mes_peer_sf_6um = Column(Float)
    nprod_sf_act = Column(Integer)
    nprod_sf_3u = Column(Integer)
    nprod_sf_6u = Column(Integer)
    mesescte_csf = Column(Integer)
    mesescte_mes = Column(Integer)
    fingreso_csf = Column(Date)
    fingreso_mes = Column(Date)
    otrosprod_csf_act = Column(Integer)
    lineaotroprod_csf_act = Column(Float)
    otrosprod_csf_3um = Column(Integer)
    lineaotroprod_csf_3um = Column(Float)
    otrosprod_csf_6um = Column(Integer)
    lineaotroprod_csf_6um = Column(Float)
    ventas_mensual = Column(Float)
    patrimonio = Column(Float)
    tipolocal = Column(String)
    tipoactividad = Column(String)
    actividadeconomica = Column(String)
    tipovivienda = Column(String)
    r_cuotapagada = Column(Float)
    NodeID = Column(Integer)
    PredictedValue = Column(Integer)
    PredictedProbability_1 = Column(Float)
    PredictedProbability_2 = Column(Float)
    SampleAssignment = Column(String)

    def __repr__(self):
        return f"<NegociosDos(numdoc='{self.numdoc}', departamento='{self.departamento}')>"

C:\Users\113167\Anaconda3\lib\site-packages\sqlalchemy\ext\declarative\clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.NegociosUno, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)


In [17]:
# Asumimos que 'negocios_dos' tiene la misma estructura; ajusta si es necesario
class NegociosDos(NegociosUno):
    __tablename__ = 'negocios_dos'

NoForeignKeysError: Can't find any foreign key relationships between 'negocios_uno' and 'negocios_dos'.

In [ ]:
# Reflexionar metadatos para tablas existentes
metadata = db.MetaData()
metadata.reflect(bind=engine)
print("Conexión y metadatos configurados correctamente.")

# Realización de operaciones CRUD (Create, Read, Update, Delete) desde Python

**Aspectos teóricos:** Las operaciones CRUD son fundamentales en la gestión de bases de datos relacionales. Representan:
- **Create**: Insertar nuevos registros.
- **Read**: Consultar y recuperar datos.
- **Update**: Modificar registros existentes.
- **Delete**: Eliminar registros.

En SQLAlchemy, estas operaciones se realizan a través de sesiones ORM, lo que asegura atomicidad y consistencia (propiedades ACID). Esto evita errores comunes como inyecciones SQL al usar parámetros vinculados.

## Create: Insertar un nuevo registro

In [ ]:
# Crear un nuevo registro en 'negocios_uno'
nuevo_negocio = NegociosUno(
    numdoc='123456789',  # Clave primaria única
    departamento='Lima',
    edad=30,
    monto_mes_ini=1000.0,
    # Agrega otros campos según sea necesario; valores por defecto o nulos para el resto
)

In [ ]:
session.add(nuevo_negocio)
session.commit()
print("Nuevo registro creado.")

## Read: Leer registros

In [ ]:
# Leer los primeros 5 registros
resultados = session.query(NegociosUno).limit(5).all()
for row in resultados:
    print(f"Numdoc: {row.numdoc}, Departamento: {row.departamento}")

## Update: Actualizar un registro

In [ ]:
# Actualizar un registro existente
registro = session.query(NegociosUno).filter_by(numdoc='123456789').first()
if registro:
    registro.departamento = 'Arequipa'
    session.commit()
    print("Registro actualizado.")
else:
    print("Registro no encontrado.")

## Delete: Eliminar un registro

In [ ]:
# Eliminar el registro
registro = session.query(NegociosUno).filter_by(numdoc='123456789').first()
if registro:
    session.delete(registro)
    session.commit()
    print("Registro eliminado.")
else:
    print("Registro no encontrado.")

# Consultas avanzadas en SQLAlchemy (JOINs, filtrado, agrupación)

**Aspectos teóricos:** Las consultas avanzadas permiten combinar datos de múltiples tablas (JOIN), aplicar condiciones (filtrado con WHERE), y resumir información (agrupación con GROUP BY). En bases relacionales, los JOINs mantienen la integridad referencial. SQLAlchemy soporta expresiones SQL nativas y sintaxis ORM, optimizando el rendimiento mediante lazy loading y eager loading.

## Filtrado

In [ ]:
# Filtrar por edad > 30 y departamento = 'Lima'
resultados = session.query(NegociosUno).filter(and_(NegociosUno.edad > 30, NegociosUno.departamento == 'Lima')).limit(5).all()
for row in resultados:
    print(f"Numdoc: {row.numdoc}, Edad: {row.edad}")

## Agrupación y funciones agregadas

In [ ]:
# Contar negocios por departamento
resultados = session.query(NegociosUno.departamento, func.count(NegociosUno.numdoc)).group_by(NegociosUno.departamento).all()
for dept, count in resultados:
    print(f"Departamento: {dept}, Count: {count}")

## JOIN (asumiendo join en 'numdoc' entre tablas)

In [ ]:
# JOIN entre negocios_uno y negocios_dos en numdoc
resultados = session.query(NegociosUno, NegociosDos).join(NegociosDos, NegociosUno.numdoc == NegociosDos.numdoc).limit(5).all()
for uno, dos in resultados:
    print(f"Numdoc: {uno.numdoc}, Departamento Uno: {uno.departamento}, Departamento Dos: {dos.departamento}")

# Optimización de consultas en bases de datos relacionales

**Aspectos teóricos:** La optimización implica reducir el tiempo de ejecución y uso de recursos. Técnicas incluyen:
- **Índices**: Aceleran búsquedas en columnas frecuentes (O(log n) vs O(n)).
- **EXPLAIN/ANALYZE**: Analiza planes de ejecución.
- **Limitar resultados**: Usar LIMIT/OFFSET para paginación.
- **Caching**: Almacenar resultados frecuentes.
- **Normalización**: Evitar redundancias para reducir I/O.

En SQLite, los índices son cruciales para tablas grandes.

## Crear un índice para optimización

In [ ]:
# Crear índice en 'departamento' para acelerar filtros
Index('idx_departamento', NegociosUno.departamento).create(bind=engine)
print("Índice creado en 'departamento'.")

## Analizar una consulta (EXPLAIN)

In [ ]:
# Ejecutar EXPLAIN en una consulta
explain_result = session.execute(text("EXPLAIN QUERY PLAN SELECT * FROM negocios_uno WHERE departamento = 'Lima'"))
for row in explain_result:
    print(row)

# Ejercicio práctico: Optimización y consulta de grandes volúmenes de datos

**Aspectos teóricos y prácticos:** Para grandes volúmenes, optimiza evitando full table scans. Usa índices, subconsultas eficientes y batch processing. Aquí, simulamos una consulta grande: contar y promediar montos por departamento, optimizando con índice.

## Consulta sin optimización (asumir full scan)

In [ ]:
start_time = datetime.now()
resultados = session.query(NegociosUno.departamento, func.count(NegociosUno.numdoc), func.avg(NegociosUno.monto_mes_ini)).group_by(NegociosUno.departamento).all()
end_time = datetime.now()
print(f"Tiempo sin índice adicional: {end_time - start_time}")
for dept, count, avg in resultados:
    print(f"Departamento: {dept}, Count: {count}, Avg Monto: {avg}")

## Agregar índice en 'monto_mes_ini' y re-ejecutar

In [ ]:
Index('idx_monto_mes_ini', NegociosUno.monto_mes_ini).create(bind=engine)
print("Índice creado en 'monto_mes_ini'.")

In [ ]:
start_time = datetime.now()
resultados = session.query(NegociosUno.departamento, func.count(NegociosUno.numdoc), func.avg(NegociosUno.monto_mes_ini)).group_by(NegociosUno.departamento).all()
end_time = datetime.now()
print(f"Tiempo con índice: {end_time - start_time}")
for dept, count, avg in resultados:
    print(f"Departamento: {dept}, Count: {count}, Avg Monto: {avg}")

In [ ]:
# ## Cerrar la sesión
session.close()
print("Sesión cerrada.")